In [1]:

import os
import json

import boto3
import sagemaker
from sagemaker.huggingface import get_huggingface_llm_image_uri, HuggingFaceModel

from dotenv import load_dotenv, find_dotenv

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\jacob\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

True

# Enviroment Configuration

## Load ACCESS Keys 

In [16]:

HF_AUTH = os.environ.get("HUGGING_FACE_AUTH")
os.environ['AWS_ACCESS_KEY_ID'] = os.environ.get("AWS_ACCESS_KEY")
os.environ['AWS_SECRET_ACCESS_KEY']= os.environ.get("AWS_SECRET_ACCESS_KEY")
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

## Set up AWS Role

In [25]:

iam = boto3.client('iam')
role = iam.get_role(RoleName='SageMakerExecutionRag')['Role']['Arn']